In [1]:
import pandas as pd
import numpy as np
import pickle
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem.porter import *
import string
import re
#from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as VS
from textstat.textstat import *
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline

ModuleNotFoundError: No module named 'textstat'

## Tweets

In [4]:
import pandas as pd

df = pd.read_csv('./data/hate-speed/haspeede_TW-train.tsv',sep='\t', names=['ID','Text','Label'])
df

,ID,Text,Label
0,576268275560116224,@matteorenzi ...all'invasione di questi animal...,1
1,815833978691260416,"È terrorismo anche questo, per mettere in uno ...",0
2,815112299044372480,@Nanoalto @FedeAngeli infatti finché ci hanno ...,0
3,577451440047853568,@dinofarnesi Piovegovernolad In Italia a...,1
4,844921803575382016,#londonattack chiedete ai buonisti del cavolo ...,1
5,811099342593462272,"Corriere: Tangenti, Mafia Capitale dimenticata...",0
6,833323806873833473,"@Pontifex_it ad uno ad uno, perché quando i mi...",0
7,835493194796314624,Il divertimento del giorno? Trovare i patrioti...,0
8,839605324684361730,Modena: Comune paga la benzina ai nomadi che p...,0
9,851906322283016192,@Cartabiancarai3 @KhalidChaouki3 altro che isl...,0


## STOPWORDS

In [4]:
from string import punctuation
import pandas as pd
#from gensim.parsing.preprocessing import STOPWORDS

print("GENSIM")
print("Len")
#len(STOPWORDS)

print("EVALITA")
pd.read_csv('./data/stopwords/IT-EVALITA16-stopwords.txt')

print("ISO")
pd.read_csv('./data/stopwords/IT-stopwordsISO-stopwords.txt')

GENSIM
Len
EVALITA
ISO


,a
0,abbastanza
1,abbia
2,abbiamo
3,abbiano
4,abbiate
5,accidenti
6,ad
7,adesso
8,affinche
9,agl


# word2vec visualization

In [1]:
def tsne_plot(model):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []

    for word in model.wv.vocab:
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

In [5]:
import gensim

word2vec_model = gensim.models.Word2Vec.load("data/word2vec/glove_WIKI")




In [7]:
from sklearn.manifold import TSNE

tsne_plot(word2vec_model)

/home/thenhz/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


MemoryError: 

In [109]:
import numpy as np

y = np.random.randint(0,2,3)
print(y)
a = np.int8(np.logical_not(y))
print(a)
np.vstack((y,a)).T

[1 1 0]
[0 0 1]


array([[1, 0],
       [1, 0],
       [0, 1]])